In [ ]:
import pickle
import time
import sys
import os
import random
import statistics
from typing import List, Set

class KellerBBSolver:
    TARGET_MAP = {3: 5, 4: 12, 5: 28, 6: 60, 7: 124}

    def __init__(self, dimension: int):
        self.DIMENSION = dimension
        self.TARGET_SIZE = self.TARGET_MAP.get(dimension, 0)
        
        filename = f'keller_codes_d{self.DIMENSION}.data'
        if not os.path.exists(filename):
            print(f"❌ Error: {filename} not found.")
            sys.exit(1)
            
        with open(filename, 'rb') as f:
            self.codes_int = pickle.load(f)
        
        self.n = len(self.codes_int)
        self.reset_stats()

    def reset_stats(self):
        """Resets counters for a new run within the batch"""
        self.count_recursive_calls = 0
        self.neighbor_checks = 0
        self.best_clique = []
        self.max_size = 0
        self.target_reached = False
        self.start_time = 0

    def are_neighbors(self, u: int, v: int) -> bool:
        self.neighbor_checks += 1
        if u == v: return False
        a, b = (u, v) if u < v else (v, u)
        return ((self.codes_int[a] >> b) & 1) == 0

    def solve(self):
        self.reset_stats()
        self.start_time = time.perf_counter()
        
        root_vertex = random.randint(0, self.n - 1)
        
        R = [root_vertex]
        P = {u for u in range(self.n) if self.are_neighbors(root_vertex, u)}
        X = {u for u in range(self.n) if u < root_vertex and self.are_neighbors(root_vertex, u)}
        
        try:
            self._bron_kerbosch_bb(R, P, X)
            if not self.target_reached:
                self._bron_kerbosch_bb([], set(range(self.n)), set())
        except KeyboardInterrupt:
            return None

        duration = time.perf_counter() - self.start_time
        return {
            'runtime': duration,
            'calls': self.count_recursive_calls,
            'clique': sorted(self.best_clique),
            'success': self.target_reached
        }

    def _bron_kerbosch_bb(self, R: List[int], P: Set[int], X: Set[int]):
        if self.target_reached: return
        self.count_recursive_calls += 1
        
        if not P and not X:
            if len(R) > self.max_size:
                self.max_size = len(R)
                self.best_clique = R.copy()
                if self.max_size >= self.TARGET_SIZE:
                    self.target_reached = True
            return

        if len(R) + len(P) <= self.max_size:
            return

        pivot_pool = list(P | X)
        pivot = max(pivot_pool[:50], key=lambda u: sum(1 for v in P if self.are_neighbors(u, v)))
        remaining_p = sorted([v for v in P if not self.are_neighbors(v, pivot)])

        for v in remaining_p:
            if self.target_reached: return
            new_P = {u for u in P if self.are_neighbors(v, u)}
            new_X = {u for u in X if self.are_neighbors(v, u)}
            self._bron_kerbosch_bb(R + [v], new_P, new_X)
            P.remove(v)
            X.add(v)

def run_batch(dimension, num_runs):
    solver = KellerBBSolver(dimension)
    results = []
    
    print(f"🧪 Starting Batch: {num_runs} runs for K({dimension})")
    print("-" * 40)

    for i in range(num_runs):
        res = solver.solve()
        if res is None: break
        results.append(res)
        print(f"Run {i+1:3d}: {res['runtime']:.4f}s | Clique Size: {len(res['clique'])}")

    if not results: return

    # Statistics Calculation
    runtimes = [r['runtime'] for r in results]
    calls = [r['calls'] for r in results]
    successes = [r['success'] for r in results]
    unique_cliques = len(set(tuple(r['clique']) for r in results))

    summary = (
        f"Performance Summary for Keller-{dimension} (Bron-Kerbosch)\n"
        f"{'='*60}\n"
        f"Search Metric          Value         | Efficiency Metric      Value\n"
        f"{'-'*60}\n"
        f"Total Runs:            {len(results):<13} | Mean Runtime:          {statistics.mean(runtimes):.4f} s\n"
        f"Success Rate:          {sum(successes)/len(results)*100:.1f}%         | Median Runtime:        {statistics.median(runtimes):.4f} s\n"
        f"Unique Cliques:        {unique_cliques}/{len(results)}      | Min Runtime:           {min(runtimes):.4f} s\n"
        f"Mean Rec. Calls:       {int(statistics.mean(calls)):<13,} | Max Runtime:           {max(runtimes):.4f} s\n"
        f"{'='*60}\n"
    )

    print("\n" + summary)
    
    # Save to file
    filename = f"keller{dimension}_batch_stats.txt"
    with open(filename, "w") as f:
        f.write(summary)
        f.write("\nDetailed Clique List:\n")
        for i, r in enumerate(results):
            f.write(f"Run {i+1}: {r['clique']}\n")
    
    print(f"✅ Statistics saved to {filename}")

if __name__ == "__main__":
    # Define your batch here
    DIM = 4
    RUNS = 100
    run_batch(DIM, RUNS)